<a href="https://colab.research.google.com/github/DayByDayBy/whypy/blob/main/why_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#   whyPy
---
### a Jupyter Notebook and LLM botherer



playing with iterative invocation of large language models and collecting the responses

(altho right now it's just a skeleton, prep)


In [ ]:
#@title installations

# installations required to run the notebook

!pip install argparse pandas sentence_transformers langchain_community langchain langchain_core datetime matplotlib

In [ ]:
#@title imports

# imports to run first (some common, some less so, may be possible to skip this step depending

import pandas as pd
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer
from langchain_community.llms import Ollama
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from datetime import datetime
import time

In [ ]:
#@title ipy input

from ipywidgets import IntText, interact

iterations = IntText(value=10, description='Iterations:')
interact(lambda iterations: iterations, iterations=iterations)

In [ ]:
#@title config and setup

impress_prompt = "quickly and concisely convince me you're the most interesting person I have met"
impress_re_prompt = "show me you're more interesting than this person: "
modelName = "llama3"
time_stamp = datetime.now().strftime("%Y%m%d_%H%M")
output_filename = f'whyPy_output_{modelName}_{time_stamp}.txt'


In [ ]:
#@title main function def

%%time

llm = Ollama(model=modelName)

def iterative_invocation(initial_prompt, re_prompt_base, max_iterations):
    latest_response = ''
    responses = []
    iteration_count = 0
    response = llm.invoke(initial_prompt)
    responses.append(response)
    latest_response = response
    iteration_count += 1

    while iteration_count < max_iterations:
        re_prompt = re_prompt_base + latest_response
        response = llm.invoke(re_prompt)
        responses.append((iteration_count, response))
        iteration_count += 1
        print("hang on, I'm still iterating!")

    return responses

def save_responses(responses, filename):
    with open(filename, 'w') as output_file:
        for index, response in responses:
            output_file.write(f'iteration {index}: \n {response}\n\n')
    print(f"Responses saved to {filename}")


In [ ]:
#@title execution

start_time = time.time()

responses = iterative_invocation(impress_prompt, impress_re_prompt, iterations.value)
indexed_responses = [(i, respo) for i, respo in enumerate(responses)]

end_time = time.time()
execution_time = end_time - start_time
print(f"Total execution time: {execution_time} seconds")

save_responses(indexed_responses, output_filename)


In [ ]:
# dataframe and display

indexed_responses_df = pd.DataFrame(indexed_responses, columns=['Iteration', 'Response'])
display(indexed_responses_df)
